In [9]:
from docx import Document
import numpy as np
from transformers import BertModel, BertTokenizer
import torch
import json

# Путь к файлу
infile = 'sample2.docx'

# Открываем документ
doc = Document(infile)

# Инициализация токенизатора и модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Сбор текста, таблиц и списков
all_text = []
structured_tables = []  # Для таблиц
structured_lists = []   # Для списков
list_checker = True     # Если есть пробел делаем False

def is_list_paragraph(paragraph):
    """
    Проверяет, является ли параграф частью списка (на основе стилей).
    """
    style_name = paragraph.style.name.lower()
    return "list" in style_name or "bullet" in style_name or "number" in style_name

current_list = []  # Временное хранилище для текущего списка

for paragraph in doc.paragraphs:
    text = paragraph.text.strip()
    
    # Проверяем, является ли параграф списком
    if is_list_paragraph(paragraph):
        if text:  # Добавляем элемент в текущий список
            current_list.append(text)
    else:
        # Если список закончился, сохраняем его
        if current_list:
            structured_lists.append(current_list)
            all_text.extend(current_list)  # Добавляем текст списка в общий массив
            current_list = []
        # Обрабатываем обычный параграф
        if text:
            all_text.append(text)

# Добавляем последний список, если он остался
if current_list:
    structured_lists.append(current_list)
    all_text.extend(current_list)

# Обработка таблиц
for table in doc.tables:
    table_data = []
    for row in table.rows:
        row_data = [cell.text.strip() for cell in row.cells]
        table_data.append(row_data)
    structured_tables.append({"data": table_data})

    # Конвертируем строки таблиц в текст
    table_text = [" | ".join(row) for row in table_data]
    all_text.extend(table_text)

# Объединяем текст для токенизации
combined_text = "\n".join(all_text)

# Токенизация текста
inputs = tokenizer(combined_text, return_tensors="pt", truncation=True, padding=True, max_length=512)

# Получение эмбеддингов из BERT
with torch.no_grad():
    outputs = model(**inputs)

# Вывод скрытых состояний [CLS]-токена
cls_embedding = outputs.last_hidden_state[:, 0, :].numpy()

# Сохранение списков и таблиц в JSON
output_data = {
    "lists": structured_lists,
    "tables": structured_tables
}

with open("document_structure.json", "w", encoding="utf-8") as f:
    json.dump(output_data, f, ensure_ascii=False, indent=4)

print("Эмбеддинги текста из документа:")
print(cls_embedding)

print("Структура документа сохранена в 'document_structure.json'")

Эмбеддинги текста из документа:
[[-4.10405308e-01 -6.94851950e-02 -7.57772923e-02 -2.38524035e-01
  -3.08586031e-01 -8.36863965e-02  9.20487523e-01  9.26771909e-02
  -2.37174094e-01 -3.72097343e-01 -1.88549608e-01  2.17751786e-01
  -6.51654601e-01  3.69204283e-02  3.16133678e-01  1.32408947e-01
   3.18059400e-02  1.56854123e-01  3.91951144e-01  3.95718962e-01
  -4.85241190e-02 -3.09472620e-01  7.46723451e-03 -3.03246230e-01
   5.64861536e-01  2.14372575e-03 -3.68815154e-01 -2.57322975e-02
  -3.96269232e-01  4.87686843e-01 -1.74444690e-01  2.65774697e-01
  -4.60526854e-01 -2.06296742e-01  2.72086203e-01 -1.91361859e-01
   5.79198122e-01 -2.11554542e-01  4.24108565e-01  1.36964202e-01
   6.72119185e-02  2.77977943e-01 -2.73852013e-02 -5.58596075e-01
  -4.24071074e-01  1.10681616e-02 -3.96519136e+00  1.41577452e-01
  -4.34443712e-01 -3.77787054e-01  3.03700835e-01 -2.43461549e-01
  -7.04258561e-01  2.55834460e-01  1.76907182e-01  4.29690599e-01
  -4.53223318e-01  2.59177797e-02  6.7582339